# Lesson 7: Updating and Deleting Data

**Duration:** 20 minutes  
**Prerequisites:** Complete Lessons 1-6  
**Learning Mode:** Read explanations, then run each SQL query

---

## �� Learning Objectives

By the end of this lesson, you will be able to:
- Update existing records with UPDATE
- Delete records safely with DELETE
- Understand the critical importance of WHERE clauses
- Use transactions for safety
- Apply database constraints
- Recover from mistakes
- Follow best practices for data modification


## ⚠️ IMPORTANT WARNING

**UPDATE and DELETE are powerful and potentially dangerous!**

### Without a WHERE clause, these commands affect EVERY row in the table!

**Examples of Dangerous Commands:**

```sql
UPDATE characters SET name = 'Bob'  -- ❌ ALL characters named Bob!
DELETE FROM characters              -- ❌ ALL characters deleted!
```

### Always Follow This Process:

1. **SELECT first** - Test your WHERE condition
2. **Double-check** - Verify you're targeting the right rows
3. **Then UPDATE/DELETE** - Change SELECT to UPDATE/DELETE
4. **Verify after** - Check the results

**Think of it like this:** UPDATE/DELETE are like using a power tool. You wouldn't use a chainsaw without checking where it's aimed!


## 🛠️ Setup: Connect to Database

**Run the next 3 cells:**


In [ ]:
# Load SQL magic extension
%load_ext sql

# Fix prettytable compatibility issue
import prettytable
try:
    # Try to access DEFAULT to see if it exists
    _ = prettytable.DEFAULT
except AttributeError:
    # If it doesn't exist, add it using SINGLE_BORDER
    from prettytable import SINGLE_BORDER
    prettytable.DEFAULT = SINGLE_BORDER

# Configure SQL magic settings
%config SqlMagic.autopandas = False
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

In [ ]:
# Connect to the Star Wars database stored in database/
%sql sqlite:///database/starwars.db

## 📝 Part 1: The UPDATE Statement

UPDATE modifies existing data in a table.

### UPDATE Syntax

```sql
UPDATE table_name
SET column1 = value1, column2 = value2, ...
WHERE condition;
```

**Parts:**
- `UPDATE table_name` - Which table to modify
- `SET column = value` - What to change
- `WHERE condition` - Which rows to affect (CRITICAL!)

### Query 1: Check Current Data Before Updating

**Always SELECT first to see what you're about to change!**


In [ ]:
%%sql
-- Query 1: Check R2-D2's current affiliation
SELECT id, name, affiliation FROM characters WHERE name = 'R2-D2';


### Query 2: Update a Single Record

Now update R2-D2's affiliation:


In [ ]:
%%sql
-- Query 2: Update R2-D2's affiliation
UPDATE characters
SET affiliation = 'Rebel Alliance'
WHERE name = 'R2-D2';


### Query 3: Verify the Change


In [ ]:
%%sql
-- Query 3: Verify R2-D2's new affiliation
SELECT id, name, affiliation FROM characters WHERE name = 'R2-D2';


**Best Practice Pattern:**
1. SELECT before (Query 1)
2. UPDATE (Query 2)
3. SELECT after (Query 3)

This confirms your change worked as expected!


### Query 4: Update Multiple Columns

You can update multiple columns in one statement:


In [ ]:
%%sql
-- Query 4: Update multiple columns at once for Darth Vader
UPDATE characters
SET species = 'Human (Cyborg)', 
    affiliation = 'Galactic Empire'
WHERE name = 'Darth Vader';


In [ ]:
%%sql
-- Verify Vader's changes
SELECT name, species, affiliation FROM characters WHERE name = 'Darth Vader';


### Query 5: Update Multiple Records

**Technique:** WHERE clause matches multiple rows.


In [ ]:
%%sql
-- Query 5: Check which droids will be updated
SELECT name, species, affiliation FROM characters WHERE species = 'Droid';


In [ ]:
%%sql
-- Update all droids
UPDATE characters
SET affiliation = 'No Affiliation'
WHERE species = 'Droid';


In [ ]:
%%sql
-- Verify all droids were updated
SELECT name, species, affiliation FROM characters WHERE species = 'Droid';


### Query 6: Update Using Calculations

**Cool feature:** You can perform math in UPDATE statements.


In [ ]:
%%sql
-- Query 6: Add 5 cm to everyone's height (growth spurt!)
UPDATE characters
SET height = height + 5
WHERE height IS NOT NULL;


In [ ]:
%%sql
-- View updated heights
SELECT name, height FROM characters ORDER BY height DESC;


**Explanation:**
- `height = height + 5` - Take current height, add 5, store result
- `WHERE height IS NOT NULL` - Only update characters with height data (avoid NULL errors)


### Query 7: Conditional Updates with CASE

**Advanced:** Use CASE for different updates based on conditions (like if-else):


In [ ]:
%%sql
-- Query 7: Update affiliations based on multiple conditions
UPDATE characters
SET affiliation = CASE
    WHEN species = 'Droid' THEN 'No Affiliation'
    WHEN species = 'Wookiee' THEN 'Rebel Alliance'
    WHEN name IN ('Obi-Wan Kenobi', 'Yoda', 'Ahsoka Tano') THEN 'Jedi Order'
    ELSE affiliation
END;


In [ ]:
%%sql
-- View results of conditional update
SELECT name, species, affiliation FROM characters ORDER BY species;


**Explanation:**

```sql
CASE
    WHEN condition1 THEN value1
    WHEN condition2 THEN value2
    ELSE default_value
END
```

Like a programming if-else statement, but in SQL!


## 🗑️ Part 2: The DELETE Statement

DELETE removes rows from a table **permanently**.

### DELETE Syntax

```sql
DELETE FROM table_name
WHERE condition;
```

### ⚠️ Critical Warning About DELETE

```sql
-- DANGEROUS! Deletes EVERYTHING:
DELETE FROM characters;

-- SAFE: Deletes specific row:
DELETE FROM characters WHERE id = 99;
```

**There is NO undo button!** (unless you use transactions, which we'll cover)


### Query 8: Add a Test Character to Delete

First, let's add a character we can safely delete:


In [ ]:
%%sql
-- Query 8: Add a test character
INSERT INTO characters (name, species, homeworld) VALUES ('Temporary Test', 'Test Species', 'Unknown');


### Query 9: Check the Test Character Exists


In [ ]:
%%sql
-- Query 9: Verify test character exists
SELECT * FROM characters WHERE name = 'Temporary Test';


### Query 10: Delete the Test Character


In [ ]:
%%sql
-- Query 10: Delete the test character
DELETE FROM characters
WHERE name = 'Temporary Test';


### Query 11: Verify Deletion


In [ ]:
%%sql
-- Query 11: Confirm the character is gone
SELECT * FROM characters WHERE name = 'Temporary Test';


**Should return 0 rows!**


### Query 12: Delete with Multiple Conditions

**Safer:** Use multiple conditions to be more precise.


In [ ]:
%%sql
-- Query 12: Add another test character
INSERT INTO characters (name, species, homeworld) VALUES ('Another Test', 'Test Species', 'Unknown');


In [ ]:
%%sql
-- Check it exists
SELECT * FROM characters WHERE species = 'Test Species';


In [ ]:
%%sql
-- Delete using multiple conditions
DELETE FROM characters
WHERE name = 'Another Test' AND species = 'Test Species';


In [ ]:
%%sql
-- Verify deletion
SELECT COUNT(*) AS remaining_test_characters FROM characters WHERE species = 'Test Species';


### Query 13: Delete Based on Subquery

**Advanced:** Delete based on related table data.

**Example:** Delete characters from unknown planets:


In [ ]:
%%sql
-- Query 13: First, see which characters would be affected
SELECT name, homeworld_id 
FROM characters 
WHERE homeworld_id IN (SELECT id FROM planets WHERE name = 'Unknown');


**Note:** We're just showing the technique. Don't actually delete these characters unless you want to!

```sql
-- If you wanted to delete them:
-- DELETE FROM characters
-- WHERE homeworld_id IN (SELECT id FROM planets WHERE name = 'Unknown');
```


## 🔒 Part 3: Data Integrity and Constraints

Constraints are rules that ensure data quality and prevent errors.

### Common Constraints

| Constraint | Purpose | Example |
|------------|---------|---------|
| `PRIMARY KEY` | Unique identifier | `id INTEGER PRIMARY KEY` |
| `NOT NULL` | Must have a value | `name TEXT NOT NULL` |
| `UNIQUE` | No duplicates allowed | `email TEXT UNIQUE` |
| `CHECK` | Must meet condition | `CHECK(height > 0)` |
| `FOREIGN KEY` | Must reference valid record | `FOREIGN KEY (homeworld_id)` |
| `DEFAULT` | Default value if none provided | `DEFAULT 'Unknown'` |

### Query 14: Testing NOT NULL Constraint

**This should fail** because name is required:


In [ ]:
%%sql
-- Query 14: Try to insert without a name (should fail)
-- Uncomment to test:
-- INSERT INTO characters (species, homeworld) VALUES ('Human', 'Earth');

-- Instead, let's check the table structure
PRAGMA table_info(characters);


**Look for:** `notnull` column shows which fields are required.

The query above is commented out because it would cause an error. That's the point - constraints protect your data!


### Query 15: Foreign Key Constraints

Foreign keys ensure relationships stay valid.

**Enable foreign keys in SQLite:**


In [ ]:
%%sql
-- Query 15: Enable foreign key constraints
PRAGMA foreign_keys = ON;


In [ ]:
%%sql
-- Check foreign key status
PRAGMA foreign_keys;


**Note:** SQLite has foreign keys disabled by default. Enabling them prevents invalid references:

```sql
-- This would fail with foreign keys ON:
-- UPDATE characters 
-- SET homeworld_id = 9999 
-- WHERE name = 'Luke Skywalker';
```

Because planet ID 9999 doesn't exist!


## 🔄 Part 4: Transactions

Transactions let you group multiple operations and **rollback if something goes wrong**.

### Transaction Syntax

```sql
BEGIN TRANSACTION;
    -- Your SQL statements here
    -- If successful:
    COMMIT;
    -- If error or changed your mind:
    -- ROLLBACK;
```

### When to Use Transactions:

- Testing risky UPDATE/DELETE operations
- Making multiple related changes
- When you want an "undo" option
- Ensuring data consistency

### Query 16: Transaction Example


In [ ]:
%%sql
-- Query 16: Start a transaction
BEGIN TRANSACTION;

-- Make test changes
UPDATE characters SET affiliation = 'Transaction Test' WHERE species = 'Human';

-- Check the changes (they're not saved yet!)
SELECT name, affiliation FROM characters WHERE species = 'Human';


### Query 17: Rollback the Transaction

**Undo everything** since BEGIN TRANSACTION:


In [ ]:
%%sql
-- Query 17: Rollback (undo) the changes
ROLLBACK;


In [ ]:
%%sql
-- Verify rollback worked - affiliations should be back to original
SELECT name, affiliation FROM characters WHERE species = 'Human';


**Magic!** The changes disappeared. It's like they never happened.

### Query 18: Transaction with Commit


In [ ]:
%%sql
-- Query 18: Start a new transaction
BEGIN TRANSACTION;

-- Add a test character
INSERT INTO characters (name, species, homeworld) VALUES ('Transaction Character', 'Test', 'Unknown');

-- Check it exists
SELECT * FROM characters WHERE name = 'Transaction Character';


In [ ]:
%%sql
-- This time, COMMIT to save the changes permanently
COMMIT;


In [ ]:
%%sql
-- Character still exists after commit
SELECT * FROM characters WHERE name = 'Transaction Character';


In [ ]:
%%sql
-- Clean up: delete the test character
DELETE FROM characters WHERE name = 'Transaction Character';


## 🎓 Practice Exercises

### Exercise 1: Safe Update with Verification


In [ ]:
%%sql
-- Exercise 1: Check Yoda's current height
SELECT name, height FROM characters WHERE name = 'Yoda';


In [ ]:
%%sql
-- Add 10 to Yoda's height
UPDATE characters
SET height = height + 10
WHERE name = 'Yoda';


In [ ]:
%%sql
-- Verify the change
SELECT name, height FROM characters WHERE name = 'Yoda';


### Exercise 2: Conditional Update with JOIN

Update characters from Tatooine:


In [ ]:
%%sql
-- Exercise 2: Check current Tatooine characters
SELECT c.name, p.name AS homeworld, c.affiliation
FROM characters c
JOIN planets p ON c.homeworld_id = p.id
WHERE p.name = 'Tatooine';


In [ ]:
%%sql
-- Update their affiliation
UPDATE characters
SET affiliation = 'Desert Natives'
WHERE homeworld_id = (SELECT id FROM planets WHERE name = 'Tatooine');


In [ ]:
%%sql
-- Verify the update
SELECT c.name, p.name AS homeworld, c.affiliation
FROM characters c
JOIN planets p ON c.homeworld_id = p.id
WHERE p.name = 'Tatooine';


### Exercise 3: Safe Deletion Practice


In [ ]:
%%sql
-- Exercise 3: Add a test character
INSERT INTO characters (name, species, homeworld) VALUES ('Delete Me', 'Test', 'Unknown');


In [ ]:
%%sql
-- Verify it exists
SELECT * FROM characters WHERE name = 'Delete Me';


In [ ]:
%%sql
-- Delete it
DELETE FROM characters WHERE name = 'Delete Me';


In [ ]:
%%sql
-- Confirm deletion (should return 0 rows)
SELECT * FROM characters WHERE name = 'Delete Me';


## 🐛 Common Errors & Troubleshooting

### Error: "syntax error"

**Problem:** Missing quotes, incorrect column name, or malformed query.

**Wrong:**
```sql
UPDATE characters SET name = Luke WHERE id = 1;  -- ❌ Missing quotes around 'Luke'
```

**Correct:**
```sql
UPDATE characters SET name = 'Luke' WHERE id = 1;  -- ✅
```

### Disaster: Updated/Deleted Everything!

**Problem:** Forgot WHERE clause!

**Prevention:**
1. **Always use WHERE** (unless you genuinely want to affect all rows)
2. **Test with SELECT first**
3. **Use transactions** for risky operations

**Recovery Options:**
1. If in transaction: `ROLLBACK`
2. Restore from backup
3. Re-run INSERT statements
4. Learn from the mistake!

### Can't Delete Due to Foreign Key

**Problem:** Trying to delete a record that other records reference.

**Example:**
```sql
DELETE FROM planets WHERE name = 'Tatooine';
-- ❌ Fails because characters reference this planet
```

**Solutions:**
1. Delete/update referencing records first
2. Update foreign keys to NULL or different value
3. Use CASCADE delete (advanced)

### UPDATE Affects Wrong Rows

**Problem:** WHERE condition too broad.

**Prevention:**


In [ ]:
%%sql
-- ALWAYS test with SELECT first
SELECT * FROM characters WHERE species = 'Human';


Check this returns ONLY the rows you want to update, then:

```sql
UPDATE characters SET affiliation = 'Changed' WHERE species = 'Human';
```


### NULL Value Comparisons

**Wrong:**
```sql
SELECT * FROM characters WHERE affiliation = NULL;  -- ❌ Won't work
```

**Correct:**
```sql
SELECT * FROM characters WHERE affiliation IS NULL;  -- ✅
```

**Remember:** Use `IS NULL` or `IS NOT NULL`, never `= NULL`


## 🎯 Challenge Problem

**Task:** Create a series of UPDATE statements that:

1. Change all Rebel Alliance members to "New Republic"
2. Add 5 cm to the height of all characters over 180 cm tall
3. Set affiliation to "Retired" for Obi-Wan Kenobi and Yoda

**Requirements:**
- Use transactions
- Test with SELECT first
- Verify all changes
- COMMIT at the end

Try it yourself before checking the solution!


### Challenge Solution


In [ ]:
%%sql
-- Start transaction
BEGIN TRANSACTION;

-- Check current Rebel Alliance members
SELECT name, affiliation FROM characters WHERE affiliation = 'Rebel Alliance';


In [ ]:
%%sql
-- 1. Update Rebel Alliance to New Republic
UPDATE characters
SET affiliation = 'New Republic'
WHERE affiliation = 'Rebel Alliance';

-- Verify
SELECT name, affiliation FROM characters WHERE affiliation = 'New Republic';


In [ ]:
%%sql
-- 2. Check characters over 180cm
SELECT name, height FROM characters WHERE height > 180;


In [ ]:
%%sql
-- Add height to tall characters
UPDATE characters
SET height = height + 5
WHERE height > 180;

-- Verify
SELECT name, height FROM characters WHERE height > 185 ORDER BY height;


In [ ]:
%%sql
-- 3. Retire the old Jedi masters
UPDATE characters
SET affiliation = 'Retired'
WHERE name IN ('Obi-Wan Kenobi', 'Yoda');

-- Verify
SELECT name, affiliation FROM characters WHERE affiliation = 'Retired';


In [ ]:
%%sql
-- Review all changes
SELECT name, affiliation, height FROM characters ORDER BY affiliation, name;


In [ ]:
%%sql
-- If satisfied with changes, commit
COMMIT;

-- If not satisfied, use ROLLBACK instead:
-- ROLLBACK;


## ✅ Checkpoint & Summary

### What You've Learnt

- ✅ Update single and multiple records with UPDATE
- ✅ Update multiple columns in one statement
- ✅ Delete records safely with DELETE
- ✅ Always use WHERE clause (or risk disaster!)
- ✅ Test with SELECT before UPDATE/DELETE
- ✅ Understand database constraints (PRIMARY KEY, FOREIGN KEY, NOT NULL, etc.)
- ✅ Use transactions for safety and testing
- ✅ Rollback changes when needed
- ✅ Handle NULL values correctly

### Key SQL Commands

| Command | Purpose | Example |
|---------|---------|---------|
| `UPDATE` | Modify existing records | `UPDATE characters SET height = 180` |
| `DELETE` | Remove records | `DELETE FROM characters WHERE id = 5` |
| `SET` | Specify new values | `SET name = 'New', species = 'Human'` |
| `CASE` | Conditional logic | `CASE WHEN species = 'Droid' THEN...` |
| `BEGIN TRANSACTION` | Start transaction | `BEGIN TRANSACTION;` |
| `COMMIT` | Save changes | `COMMIT;` |
| `ROLLBACK` | Undo changes | `ROLLBACK;` |
| `PRAGMA` | Configure database | `PRAGMA foreign_keys = ON;` |

### Safety Checklist

Before running UPDATE or DELETE:

- [ ] Have I included a WHERE clause?
- [ ] Did I test with SELECT first?
- [ ] Am I sure this targets the right rows?
- [ ] Do I have a backup (or using transactions)?
- [ ] Have I double-checked the conditions?
- [ ] Did I count the rows affected?

## 🎉 Excellent Work!

You can now safely modify and delete data! In the next lesson, you'll learn about advanced queries using subqueries.

**Ready to continue?** Open `lesson8_advanced.ipynb`

---

## 💾 Git Commands (for reference)

```bash
git status
git add solutions/lesson7_modifications.ipynb database/starwars.db
git commit -m "Completed Lesson 7: UPDATE and DELETE with safety practices"
git push
```
